In [1]:
!nvidia-smi

Thu Apr 13 18:57:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.09    Driver Version: 528.33       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:2B:00.0  On |                  N/A |
|  0%   40C    P0    47W / 220W |   1136MiB /  8192MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from PIL import Image
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import os

In [3]:
# Loading Dataset

name = "HRSID"
dataset_dir = os.path.abspath("Datasets/HRSID")

# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset

dataset_traintest = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name=name,
)

 100% |███████████████| 5604/5604 [30.8s elapsed, 0s remaining, 176.4 samples/s]      


In [4]:
name = "HRSID_train"
dataset_dir = os.path.abspath("Datasets/HRSID")
label_path = os.path.abspath("Datasets/HRSID/annotations/train2017.json")
# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset

dataset_train = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name=name,
    labels_path=label_path
)

 100% |███████████████| 3642/3642 [19.8s elapsed, 0s remaining, 158.1 samples/s]      


In [5]:
os.getcwd()

'/home/k3vinli/ENEE439/Capstone'

In [6]:
name = "HRSID_test"
dataset_dir = os.path.abspath("Datasets/HRSID")
label_path = os.path.abspath("Datasets/HRSID/annotations/test2017.json")
# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset

dataset_test = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name=name,
    labels_path=label_path
)

 100% |███████████████| 1962/1962 [10.5s elapsed, 0s remaining, 119.7 samples/s]     


In [7]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

In [13]:
paths = dataset_train.values("filepath")
sample = dataset_train[paths[1]]
sample.metadata

<ImageMetadata: {
    'size_bytes': None,
    'mime_type': None,
    'width': 800,
    'height': 800,
    'num_channels': None,
}>

In [21]:
sample

<Sample: {
    'id': '643888ea4f7dbf1ed831aefd',
    'media_type': 'image',
    'filepath': '/home/k3vinli/ENEE439/Capstone/Datasets/HRSID/data/P0001_0_800_8400_9200.jpg',
    'tags': [],
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 800,
        'height': 800,
        'num_channels': None,
    }>,
    'detections': <Detections: {
        'detections': [
            <Detection: {
                'id': '643888ea4f7dbf1ed831aeed',
                'attributes': {},
                'tags': [],
                'label': 'ship',
                'bounding_box': [0.3825, 0.65, 0.01625, 0.045],
                'mask': None,
                'confidence': None,
                'index': None,
                'iscrowd': 0,
            }>,
            <Detection: {
                'id': '643888ea4f7dbf1ed831aeee',
                'attributes': {},
                'tags': [],
                'label': 'ship',
                'bounding_box': [0

In [23]:
img = Image.open(paths[1]).convert("RGB")
detections = sample["detections"].detections
segmentations = sample["segmentations"].detections

In [35]:
classes = dataset_train.distinct("%s.detections.label" % "segmentations")

In [36]:
labels_map_rev = {c: i for i, c in enumerate(classes)}
labels_map_rev


{'ship': 0}

In [ ]:
for det in detections:
    category_id=labels_map_rev[det.label]
    coco_obj=fouc.COCOObject.from_label(det, sample.metadata, category_id=category_id)
    x,y,w,h=coco_obj.bbox
    boxes.append

In [24]:
segmentations

[<Detection: {
     'id': '643888ea4f7dbf1ed831aef2',
     'attributes': {},
     'tags': [],
     'label': 'ship',
     'bounding_box': [0.3825, 0.65, 0.01625, 0.045],
     'mask': array([[False,  True,  True,  True, False, False, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True,  True, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True,  True, False, False,
             False, False, False, False

In [9]:
session = fo.launch_app()

In [10]:
class HRSIDDataset(Dataset):
    """A class to construct a Pytorch dataset from a FiftyOne dataset
    """
    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="segmentations",
        classes=None):

        self.samples = fiftyone_dataset
        self.transforms = transforms 
        self.gt_field = gt_field
        
        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )
        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        # maps values to individual classes
        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}            

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []
        area = []
        iscrowd = []
        masks = []
        detections = sample[self.gt_field].detections
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_label(
                det, metadata, category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)
            masks.append(coco_obj.segmentation)

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.as_tensor([idx])
        target["area"] = torch.as_tensor(area, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)
        target["masks"] = torch.as_tensor(masks, dtype=torch.uint8)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes        